# Praca domowa 1
## Bartosz Siński 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport
import datetime as dt
from datetime import datetime  

In [ ]:
fires_df = pd.read_csv("./src/forest_fires_dataset.csv")

In [ ]:
fires_df.head()

In [ ]:
fires_df.info()

In [ ]:
fires_df.describe()

In [ ]:
fires_df.isnull().sum()

Po wstępnym przejrzeniu naszych danych widzimy, że raczej będzie problemu z brakujacymi danymi. Jednak już teraz widać, że rozkład wartości przy niektórych atrybutach będzie daleki od normalnego. 

In [ ]:
fires_subset1 = fires_df[['RH','FFMC','DMC','DC','ISI','temp','wind','rain','area']]
corrmat = fires_subset1.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

Widzimy, że najlepsze korelacje zachodzą pomiędzy poszczególnymi wyznacznikami wiglotności lasu: 
* FFMC - wilgotność ściółki
* DMC - wilgotnośc wartwy pod ściółką
* DC - wilgotnośc gleby

Nie jest to zbytnio odkrywcze jednak dobrze widac także zależnośći pomiędzy wspomnianymi wcześniej wyznacznikami, a temperaturą danego dnia i ryzykiem rozprzestrzenienia się pożaru. Przyjrzymy się bliżej temu jaśniejszemu kwadratowi.

In [ ]:
fires_subset = fires_df[['FFMC','DMC','DC','ISI','temp']]
cm = np.corrcoef(fires_subset.values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=fires_subset.columns, xticklabels=fires_subset.columns)
plt.show()

Zmiennymi najbardziej skorelowanymi są DC i DMC. Najgorzej skorelowana z resztą wydaje się być zmienna ISI jednak wyjątkiem jest FFMC, która ma z ISI ma współczynnik korelacji 0.53. 

In [ ]:
sns.set()
sns.pairplot(fires_subset, height = 5)
plt.show();

Widzimy, że dla FFMC i ISI istineją rekordy o wartościach mocno odstających

In [ ]:
isi_max = fires_subset.loc[fires_subset['ISI']==fires_subset['ISI'].max()]
isi_max

In [ ]:
ffmc_min = fires_subset.loc[fires_subset['FFMC']==fires_subset['FFMC'].min()]
ffmc_min

W większości przypadku nie podążają one z pożądanymi przez nas trendami więc usuńmy je i zobaczmy czy zależności będą wyglądać inaczej.

In [ ]:
fires_subset = fires_subset.loc[(fires_subset['ISI']!=fires_subset['ISI'].max()) & (fires_subset['FFMC']!=fires_subset['FFMC'].min())]
sns.set()
sns.pairplot(fires_subset, height = 5)
plt.show();

Najbliżej rozkładu rozmalnego wydają się być wartośći temp i ISI. Potwierdzają się także nasze przewidywania, że najlepsze zależności widać pomiędzy DMC i DC. Przyjrzyjmy się teraz histogramom ICI i temp.

In [ ]:
sns.displot(fires_subset, x="ISI",kde=True)

Widzimy, że rozkład jest leptokurtyczny i prawoskośny. 

In [ ]:
sns.displot(fires_subset, x="temp",kde=True)

Widzimy, że rozkład jest leptokurtyczny i lewośkośny. Jednak jest on najblizszy rozkładowani normalnemu z badanych przez nas zmiennych. Dodatkowo poza badanym przez nas podbiorem danych sprawdzimy zależności pomiędzy wiatrem, a współczynnikiem ISI oznaczającym ryzyko rozprzestrzeniania się, które uważamy, że pomimo niskiego współczynnika korelacji powinny być od siebie zależne.

In [ ]:
sns.scatterplot(data = fires_df,x="wind",y="ISI")

Niestety nasze założenie się nie sprawdziło i żadnej zależności nie widać. Sprawdźmy jak pomiędzy miesiącami zmieniała się pogoda (temperatura, deszcz, wiatr) i czy wpływało to na obszar spalonego lasu.

In [ ]:
fires_df_grouped = fires_df[['month','rain','area','wind','temp']].groupby('month')
fires_df_grouped = fires_df_grouped.mean()
fires_df_grouped = fires_df_grouped.reset_index()
fires_df_grouped['month'] = pd.to_datetime(fires_df_grouped['month'],format="%b")
fires_df_grouped = fires_df_grouped.sort_values(by='month') 
fires_df_grouped['rain'] = fires_df_grouped['rain'].div(fires_df_grouped['rain'].iloc[2])
fires_df_grouped['temp'] = fires_df_grouped['temp'].div(fires_df_grouped['temp'].iloc[0])
fires_df_grouped['wind'] = fires_df_grouped['wind'].div(fires_df_grouped['wind'].iloc[0])
fires_df_grouped['area'] = fires_df_grouped['area'].div(fires_df_grouped['area'].iloc[1])
fires_df_long = pd.melt(fires_df_grouped,id_vars =['month'],value_vars=['rain','area','wind','temp'])
fires_df_long['month'] = fires_df_long['month'].dt.strftime('%b')
sns.relplot(data=fires_df_long,kind="line",x="month",y='value',hue="variable")

Powyższy wykres przedstawia jak zmieniały się poszczególne wskaźniki pogodowe w danych miesiącach. Miedzy innymi widać, że w sierpniu gdy padało najwięcej deszczu, zmniejszył sie obszar spalonego lasu. Teraz sprawdźmy gdzie spłoneło go najwięcej.

In [ ]:
fires_area = fires_df[["X", "Y", "area"]]
fires_area = fires_area.groupby(["X","Y"])
fires_area = fires_area.sum()
fires_area = fires_area.reset_index()
fires_area= fires_area.pivot("X","Y","area")
fires_area= fires_area.sort_index(ascending=False)
sns.heatmap(fires_area)

X i Y to koordynaty, więc widzimy, że najwięcej pożarów było  w centrum parku. Teraz do eksploracji skorzystamy z pandas profiling:

In [ ]:
profile = ProfileReport(fires_df, title='Fires Data Exploration')
profile

Narzędzie pandas-profiling bardzo ułatwiwa eksploracje danych. Największą jego zaletą jest interaktywne szukanie zależności pomiędzy zmiennymi oraz podsumowania wraz ze szczegółami dla każdej zmiennej. Bardzo przyspieszyło to analize naszego zbioru danych. Do ograniczeń należy między innymi brak możliwości porównywania między sobą zmodyfikowanych zbiorów danych na przykład zbioru oryginalnego ze zbiorem z usuniętymi obserwacjami odstającymi. 